# Preflop Equity Analysis

Analyze preflop equity for common matchups and visualize convergence.

In [ ]:
from holdem_lab import (
    parse_cards, format_cards,
    EquityRequest, PlayerHand, calculate_equity,
)
import matplotlib.pyplot as plt

## Classic Matchups

In [ ]:
# Define classic preflop matchups
matchups = [
    ("AA vs KK", "Ah Ad", "Kh Kd"),
    ("AA vs AKs", "Ah Ad", "Ac Kc"),
    ("KK vs AKs", "Kh Kd", "Ac Kc"),
    ("JJ vs AKo", "Jh Jd", "Ac Kd"),
    ("AKs vs QQ", "Ah Kh", "Qc Qd"),
    ("AKo vs 22", "Ah Kd", "2c 2s"),
    ("AKs vs 72o", "Ah Kh", "7c 2d"),
]

print("Preflop Equity Matchups (10,000 simulations)")
print("=" * 50)

results = []
for name, hand1, hand2 in matchups:
    request = EquityRequest(
        players=[
            PlayerHand(hole_cards=tuple(parse_cards(hand1))),
            PlayerHand(hole_cards=tuple(parse_cards(hand2))),
        ],
        num_simulations=10000,
        seed=42,
    )
    result = calculate_equity(request)
    eq1 = result.players[0].equity
    eq2 = result.players[1].equity
    results.append((name, eq1, eq2))
    print(f"{name:15} | {hand1:8} {eq1:5.1%} vs {eq2:5.1%} {hand2:8}")

## Equity Bar Chart

In [ ]:
# Create bar chart
fig, ax = plt.subplots(figsize=(10, 6))

names = [r[0] for r in results]
eq1 = [r[1] * 100 for r in results]
eq2 = [r[2] * 100 for r in results]

x = range(len(names))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], eq1, width, label='Hand 1', color='steelblue')
bars2 = ax.bar([i + width/2 for i in x], eq2, width, label='Hand 2', color='coral')

ax.set_ylabel('Equity (%)')
ax.set_title('Preflop Equity Matchups')
ax.set_xticks(x)
ax.set_xticklabels(names, rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 100)
ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## Convergence Analysis

Watch how equity estimates converge as we run more simulations.

In [ ]:
# Run equity calculation with convergence tracking
request = EquityRequest(
    players=[
        PlayerHand(hole_cards=tuple(parse_cards("Ah Ad"))),
        PlayerHand(hole_cards=tuple(parse_cards("Kh Kd"))),
    ],
    num_simulations=10000,
    seed=42,
    track_convergence=True,
    convergence_interval=100,
)

result = calculate_equity(request)

# Plot convergence
if result.convergence:
    sims = [p.simulation for p in result.convergence]
    eq_aa = [p.equities[0] * 100 for p in result.convergence]
    eq_kk = [p.equities[1] * 100 for p in result.convergence]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(sims, eq_aa, label='AA', color='steelblue', linewidth=2)
    ax.plot(sims, eq_kk, label='KK', color='coral', linewidth=2)
    
    # Add reference lines for "true" values
    ax.axhline(y=82, color='steelblue', linestyle='--', alpha=0.5)
    ax.axhline(y=18, color='coral', linestyle='--', alpha=0.5)
    
    ax.set_xlabel('Simulations')
    ax.set_ylabel('Equity (%)')
    ax.set_title('AA vs KK: Equity Convergence')
    ax.legend()
    ax.set_ylim(0, 100)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nFinal equities: AA={eq_aa[-1]:.1f}%, KK={eq_kk[-1]:.1f}%")

## Three-Way Pots

In [ ]:
# Three-way preflop all-in
request = EquityRequest(
    players=[
        PlayerHand(hole_cards=tuple(parse_cards("Ah Ad"))),
        PlayerHand(hole_cards=tuple(parse_cards("Kh Kd"))),
        PlayerHand(hole_cards=tuple(parse_cards("Qh Qd"))),
    ],
    num_simulations=10000,
    seed=42,
)

result = calculate_equity(request)

print("Three-Way: AA vs KK vs QQ")
print("-" * 30)
hands = ["AA", "KK", "QQ"]
for i, p in enumerate(result.players):
    print(f"  {hands[i]}: {p.equity:.1%} equity (win: {p.win_rate:.1%}, tie: {p.tie_rate:.1%})")

## Pocket Pair vs Two Overcards

The classic "coin flip" scenario.

In [ ]:
# Various pair vs overcards matchups
pair_vs_overcards = [
    ("22 vs AKo", "2h 2d", "Ac Kd"),
    ("55 vs AKo", "5h 5d", "Ac Kd"),
    ("88 vs AKo", "8h 8d", "Ac Kd"),
    ("TT vs AKo", "Th Td", "Ac Kd"),
    ("JJ vs AKo", "Jh Jd", "Ac Kd"),
    ("QQ vs AKo", "Qh Qd", "Ac Kd"),
]

print("Pocket Pair vs AKo")
print("=" * 40)

for name, pair, overcards in pair_vs_overcards:
    request = EquityRequest(
        players=[
            PlayerHand(hole_cards=tuple(parse_cards(pair))),
            PlayerHand(hole_cards=tuple(parse_cards(overcards))),
        ],
        num_simulations=10000,
        seed=42,
    )
    result = calculate_equity(request)
    print(f"{name:12} | Pair: {result.players[0].equity:.1%} | AKo: {result.players[1].equity:.1%}")